# Filter airborne PFP data
- R Program to filter aircraft data for strong local continental influences, subtract off NOAA in situ SPO, and write out flat text files

In [9]:
library('ncdf4')
library('yaml')

In [10]:
project_tmpdir_obs = read_yaml('../_config_calc.yml')$project_tmpdir_obs
username = Sys.info()['user']
project_tmpdir_obs = gsub('\\{\\{env\\[\'USER\'\\]\\}\\}', username, project_tmpdir_obs)

In [11]:
# read in preprocessed aircraft files from read_aircraft_pfp.r
load('HIPPO_PFP.RData')
load('ATom_PFP.RData')

In [12]:
# calculate datetime variables
hippodt=ISOdatetime(hippomerge$year,hippomerge$mon,hippomerge$day,hippomerge$hour,hippomerge$min,hippomerge$sec,tz='UTC')
atomdt=ISOdatetime(atommerge$year,atommerge$mon,atommerge$day,atommerge$hour,atommerge$min,atommerge$sec,tz='UTC')

In [13]:
# read in NOAA in situ record from SPO
sponc=nc_open(paste(project_tmpdir_obs,'/obspack_co2_1_GLOBALVIEWplus_v6.0_2020-09-11/data/nc/co2_spo_surface-insitu_1_allvalid.nc',sep=''))
spoco2=data.frame(cbind(ncvar_get(sponc,'time_decimal'),t(ncvar_get(sponc,'time_components')),ncvar_get(sponc,'value')*1E6)) ; colnames(spoco2)=c('date','year','mon','day','hour','min','sec','co2')
qcflag=ncvar_get(sponc,'qcflag'); spoco2$co2[substr(qcflag,1,1)!='.']=NA; spoco2$co2[substr(qcflag,2,2)!='.']=NA
spodt=ISOdatetime(spoco2$year,spoco2$mon,spoco2$day,spoco2$hour,spoco2$min,spoco2$sec,tz='UTC')

# HIPPO

In [14]:
# filter
ints=read.table(paste(project_tmpdir_obs,'/hippo_xsect_filt_datetime.txt',sep=''),header=T) 
startdt=ISOdatetime(ints$startyear,ints$startmon,ints$startday,ints$starthour,ints$startmin,ints$startsec,tz='UTC')
stopdt=ISOdatetime(ints$stopyear,ints$stopmon,ints$stopday,ints$stophour,ints$stopmin,ints$stopsec,tz='UTC')
blfilt=rep(T,nrow(hippomerge))
for(i in c(1:nrow(ints))){
blfilt[difftime(hippodt,startdt[i])>=0&difftime(hippodt,stopdt[i])<=0]=F
}
hippodt=hippodt[blfilt]
hippomerge=hippomerge[blfilt,]
print(paste('Filtered ',sum(!blfilt),' of ',length(blfilt),' HIPPO obs (',round(sum(!blfilt)/length(blfilt)*100,1),'%)',sep=''))

[1] "Filtered 14 of 1374 HIPPO obs (1%)"


In [15]:
# calculate differences
hippomerge$co2mspo=round(hippomerge$co2-approx(as.POSIXct(spodt),spoco2$co2,as.POSIXct(hippodt))$y,3) ## co2 = 'CO2_CCG'
hippomerge$co2mqcls=round(hippomerge$co2-hippomerge$co2qcls,3)
hippomerge$co2moms=round(hippomerge$co2-hippomerge$co2oms,3)
hippomerge$co2mao2=round(hippomerge$co2-hippomerge$co2ao2,3)
hippomerge$ch4mqcls=round(hippomerge$ch4pfp-hippomerge$ch4qcls,3)

In [16]:
# write out
write(names(hippomerge),'../data/aircraft-obs/HIPPO_SO_mSPO_pfp.txt',ncol=ncol(hippomerge))
write(t(hippomerge),'../data/aircraft-obs/HIPPO_SO_mSPO_pfp.txt',ncol=ncol(hippomerge),append=T)

print(apply(!is.na(hippomerge),2,sum))

    year      mon      day     hour      min      sec     camp      flt 
    1360     1360     1360     1360     1360     1360     1360     1360 
    prof      lat      lon      alt pressure    theta      co2  co2qcls 
    1360     1360     1360     1360     1355     1355     1296      944 
  co2oms   co2ao2   ch4pfp  ch4qcls   sf6pfp    strat   h2oref   n2oref 
     995     1125     1348     1010     1347     1360     1360     1355 
   o3ref  co2mspo co2mqcls  co2moms  co2mao2 ch4mqcls 
    1350     1296      911      965     1071     1000 


# ATom

In [17]:
# filter
ints=read.table(paste(project_tmpdir_obs,'/atom_xsect_filt_datetime.txt',sep=''),header=T)
startdt=ISOdatetime(ints$startyear,ints$startmon,ints$startday,ints$starthour,ints$startmin,ints$startsec,tz='UTC')
stopdt=ISOdatetime(ints$stopyear,ints$stopmon,ints$stopday,ints$stophour,ints$stopmin,ints$stopsec,tz='UTC')
blfilt=rep(T,nrow(atommerge))
for(i in c(1:nrow(ints))){
    blfilt[difftime(atomdt,startdt[i])>=0&difftime(atomdt,stopdt[i])<=0]=F
}
atomdt=atomdt[blfilt]
atommerge=atommerge[blfilt,]
print(paste('Filtered ',sum(!blfilt),' of ',length(blfilt),' ATom obs (',round(sum(!blfilt)/length(blfilt)*100,1),'%)',sep=''))

[1] "Filtered 13 of 875 ATom obs (1.5%)"


In [18]:
# calculate differences
atommerge$co2mspo=round(atommerge$co2-approx(as.POSIXct(spodt),spoco2$co2,as.POSIXct(atomdt))$y,2) ## co2 = 'CO2_PFP'
atommerge$co2mqcls=round(atommerge$co2-atommerge$co2qcls,3)
atommerge$co2mao2=round(atommerge$co2-atommerge$co2ao2,3)
atommerge$co2mnoaa=round(atommerge$co2-atommerge$co2noaa,3)
atommerge$ch4mqcls=round(atommerge$ch4pfp-atommerge$ch4qcls,3)
atommerge$ch4mnoaa=round(atommerge$ch4pfp-atommerge$ch4noaa,3)

In [19]:
# write out
write(names(atommerge),'../data/aircraft-obs/ATOM_SO_mSPO_pfp.txt',ncol=ncol(atommerge))
write(t(atommerge),'../data/aircraft-obs/ATOM_SO_mSPO_pfp.txt',ncol=ncol(atommerge),append=T)

print(apply(!is.na(atommerge),2,sum))

    year      mon      day     hour      min      sec     camp      flt 
     862      862      862      862      862      862      862      862 
    prof      lat      lon      alt pressure    theta      co2  co2noaa 
     862      862      862      862      862      862      859      838 
 co2qcls   co2ao2   ch4pfp  ch4noaa  ch4qcls   sf6pfp    strat   h2oref 
     592      742      849      838      585      849      862      862 
  n2oref    o3ref  co2mspo co2mqcls  co2mao2 co2mnoaa ch4mqcls ch4mnoaa 
     860      862      859      591      739      835      577      826 
